### Notebook responsável por buscar os clubes participantes no campeonato.

In [10]:
import datetime
import requests
import json
import pandas as pd

In [1]:
%run ./database.ipynb
%run ./partidas.ipynb

In [4]:
def get_clubs_from_cartola_api():
    clubs_page = requests.get('https://api.cartolafc.globo.com/partidas')
    return json.loads(clubs_page.content)

In [5]:
def extract_clubs_from_json(clubs_json):
    CHAMPIONSHIP_CLUBS = 20
    clubs_dict = clubs_json['clubes']
    if (len(clubs_dict) != CHAMPIONSHIP_CLUBS):
        raise Exception('wrong_number','O campeonato deveria ter 20 times!')
        
    return clubs_dict

In [6]:
def create_clubs_dataframe(clubs_dict):
    clubs = []
    year = datetime.date.today().year

    for id in clubs_dict:
        club = {'id': clubs_dict[id]['id'],
                'name': clubs_dict[id]['nome'],
                'initials': clubs_dict[id]['abreviacao'],
                'year': year
               }
        clubs.append(club)
    
    return pd.DataFrame(clubs)

In [11]:
def collect_clubs_and_save_to_database(engine):
    clubs_json = get_clubs_from_cartola_api()
    clubs_dict = extract_clubs_from_json(clubs_json)
    df_clubs = create_clubs_dataframe(clubs_dict)
    save_data(engine, 'club', df_clubs)
    return count_data(engine, 'club').fetchall()